# LSMS Processing

Data available at [https://sft.mitre.org](https://sft.mitre.org) in Luma subfolder.

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
year = 2015

if year == 2015:
    LSMS_PATH = 'ETH_2015_ESS_v02_M_CSV_LSMS'
elif year == 2013:
    LSMS_PATH = 'ETH_2013_ESS_DTA files_LSMS'
elif year == 2011:
    LSMS_PATH = 'ETH_2011_ERSS_v02_M_CSV_LSMS'    

## Load Crop Yield Data

In [3]:
vars_ = pd.read_csv('Inputs/Harvest_by_Field_data_dict.csv').set_index('Variable')

In [4]:
vars_dict = vars_.to_dict()

In [5]:
if year in [2013, 2015]:
    vars_to_select = ['household_id2','saq01','saq02','saq03','saq04','parcel_id','field_id','crop_name','ph_s9q04_a','ph_s9q04_b']

# 2011 did not have non-standardized (kg, g) units, so we should just use the kg variable    
elif year == 2011:
    vars_to_select = ['household_id2','saq01','saq02','saq03','saq04','parcel_id','field_id','crop_name','ph_s9q12_a']

In [6]:
if year == 2015:
    int_path = "Post-Harvest/"
    wave = 3
elif year == 2013:
    int_path = ""
    wave = 2
elif year == 2011:
    int_path = ""
    wave = 1

if year == 2013:
    c_yields = pd.read_stata(f'{LSMS_PATH}/{int_path}sect9_ph_w{wave}.dta')
else:
    c_yields = pd.read_csv(f'{LSMS_PATH}/{int_path}sect9_ph_w{wave}.csv')

if year == 2013:
    c_yields['crop_name'] = c_yields['crop_code']
    
if year == 2011:
    c_yields['household_id2'] = c_yields['household_id']

In [7]:
c_yields = c_yields[vars_to_select]

In [8]:
c_yields = c_yields.rename(columns=vars_dict['Definition'])

In [9]:
c_yields = c_yields.set_index('Unique HH ID in wave 2')

Remove NA values from dataset (household missing)

In [10]:
c_yields = c_yields.dropna()

In [11]:
num_hh = c_yields.reset_index().groupby('Unique HH ID in wave 2')['Field ID'].count().shape[0]
num_fields = sum(c_yields.reset_index().groupby('Unique HH ID in wave 2')['Field ID'].count())

In [12]:
print(f"There are {num_hh} households with {num_fields} total fields.")

There are 2759 households with 25924 total fields.


## Load Field Area information

In [13]:
if year == 2015:
    int_path = "Post-Planting/"
    wave = 3
    path = "sect_3"
elif year == 2013:
    int_path = ""
    wave = 2
    path = "sect_3"    
elif year == 2011:
    int_path = ""
    wave = 1
    path = "sect3_"    

In [14]:
if year == 2013:
    c_area = pd.read_stata(f'{LSMS_PATH}/{int_path}{path}rca_pp_w{wave}.dta')
else:
    c_area = pd.read_csv(f'{LSMS_PATH}/{int_path}{path}rca_pp_w{wave}.csv')    

In [15]:
if year in [2013, 2015]:
    area_vars = ['household_id2','parcel_id','field_id','pp_rcq02_a','pp_rcq02_b']

# 2011 did not have non-standardized (kg, g) units, so we should just use the kg variable    
elif year == 2011:
    area_vars = ['household_id','parcel_id','field_id','pp_rcq02_a','pp_rcq02_b']

In [16]:
c_area = c_area[area_vars].dropna(subset=['pp_rcq02_a'])

if year == 2011:
    c_area['household_id2'] = c_area['household_id']
    del(c_area['household_id'])

In [17]:
print(f"{c_area.shape[0]} total fields have area information")

4191 total fields have area information


In [18]:
new_vars = {'parcel_id': 'Parcel ID', 'field_id': 'Field ID', 'pp_rcq02_a': 'Area', 'pp_rcq02_b': 'Area (decimal)'}
c_area = c_area.rename(columns=new_vars)

In [19]:
new_df = pd.merge(c_yields.reset_index(), c_area, 
                  how='left', 
                  left_on=['Unique HH ID in wave 2','Parcel ID','Field ID'], 
                  right_on = ['household_id2','Parcel ID','Field ID'])

In [20]:
c_yields = new_df.set_index('Unique HH ID in wave 2')

## Load Geo Houshould information

In [21]:
if year == 2015:
    int_path = "Geovariables/"
    f = "ETH_HouseholdGeovars_y3.csv"
elif year == 2013:
    int_path = ""
    f = "Pub_ETH_HouseholdGeovars_Y2.dta"
elif year == 2011:
    int_path = ""
    f = "Pub_ETH_HouseholdGeovariables_Y1.csv"    
    
if year == 2013:
    household_geo = pd.read_stata(f'{LSMS_PATH}/{int_path}{f}')
else:
    household_geo = pd.read_csv(f'{LSMS_PATH}/{int_path}{f}')    

In [22]:
if year in [2013, 2015]:
    household_vars = {
    'household_id2':'Unique HH ID in wave 2',
    'lon_dd_mod':'EA Longitude (WGS84) Modified',
    'lat_dd_mod':'EA Latitude (WGS84) Modified'
    }
elif year == 2011:
    household_vars = {
    'household_id':'Unique HH ID in wave 2',
    'LAT_DD_MOD':'EA Latitude (WGS84) Modified',
    'LON_DD_MOD':'EA Longitude (WGS84) Modified'
    }    

In [23]:
household_geo = household_geo[household_vars.keys()].rename(columns=household_vars)

Add this geospatial data to crop yield information

In [24]:
c_yields = c_yields.join(household_geo.set_index('Unique HH ID in wave 2'), on='Unique HH ID in wave 2', how='left')

In [25]:
c_yields

,Region Code,Zone Code,Woreda Code,Kebele/FA Code,Parcel ID,Field ID,Crop Name,How much [CROP] did you harvest from this [FIELD](QUANTITY),How much [CROP] did you harvest from this [FIELD](UNIT),household_id2,Area,Area (decimal),EA Longitude (WGS84) Modified,EA Latitude (WGS84) Modified
Unique HH ID in wave 2,,,,,,,,,,,,,,
1.010109e+16,1.0,1.0,1.0,16.0,1,2,MAIZE,12.0,181.0,NaN,NaN,NaN,37.890876,14.353816
1.010109e+16,1.0,1.0,1.0,16.0,1,3,RED PEPPER,2.0,1.0,1.010109e+16,8.0,30.0,37.890876,14.353816
1.010109e+16,1.0,1.0,1.0,16.0,1,4,SORGHUM,350.0,1.0,NaN,NaN,NaN,37.890876,14.353816
1.010109e+16,1.0,1.0,1.0,16.0,1,2,MAIZE,30.0,181.0,NaN,NaN,NaN,NaN,NaN
1.010109e+16,1.0,1.0,1.0,16.0,1,3,RED PEPPER,24.0,181.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.501021e+17,15.0,1.0,2.0,28.0,3,1,COFFEE,25.0,1.0,NaN,NaN,NaN,NaN,NaN
1.501021e+17,15.0,1.0,2.0,28.0,4,2,CHAT,3.0,192.0,NaN,NaN,NaN,NaN,NaN
1.501021e+17,15.0,1.0,2.0,28.0,3,1,BANANAS,2.0,42.0,NaN,NaN,NaN,NaN,NaN


In [26]:
c_yields = c_yields.reset_index()

In [27]:
c_yields.shape

(25924, 15)

### Crop Unit Conversions

In [28]:
if year == 2015:
    int_path = "Food and Crop Conversion Factors/"
    f = "Crop_CF_Wave3.csv"
elif year == 2013:
    int_path = ""
    f = "Crop_CF_Wave2.dta"
elif year == 2011:
    int_path = ""
    f = "Food_CF_Wave1.csv"    

if year == 2013:
    c_conversions = pd.read_stata(f'{LSMS_PATH}/{int_path}{f}')
else:
    c_conversions = pd.read_csv(f'{LSMS_PATH}/{int_path}{f}')

In [29]:
if year == 2015:
    u_codes = pd.read_csv('Inputs/2015_unit_codes.csv')
    c_codes = pd.read_csv('Inputs/2015_crop_codes.csv')    
    c_conversions = c_conversions\
        .join(u_codes.set_index('Unit Code'), how='left', on='unit_cd')\
        .join(c_codes.set_index('Crop Code'), how='left', on='crop_code')\
        #.rename(columns={"Unit Name": "Unit Code"})

In [30]:
if year == 2013:
    c_conversions_vars = {'crop_code': 'Crop Name', 'unit_cd': 'Unit Code', 'mean_cf_nat': 'Conversion Factor (National)'}
elif year == 2011:
    c_conversions_vars = {'item_cd': 'Crop Name', 'unit_cd': 'Unit Code', 'mean_cf_nat': 'Conversion Factor (National)'}
elif year == 2015:
    c_conversions_vars = {'Crop Name': 'Crop Name', 'unit_cd': 'Unit Code', 'Unit Name': 'Unit Name', 'mean_cf_nat': 'Conversion Factor (National)'}
    
c_conversions = c_conversions[c_conversions_vars.keys()]
c_conversions = c_conversions.rename(columns=c_conversions_vars)

In [31]:
c_conversions.head()

,Crop Name,Unit Code,Unit Name,Conversion Factor (National)
0,BARLEY,1,Kilogram,1.000000
1,BARLEY,2,Gram,0.001000
2,BARLEY,3,Quintal,100.000000
3,BARLEY,8,Jog,0.831000
4,BARLEY,21,Akumada/Dawla/Lekota Small,25.264999


In [32]:
c_yields.head()

,Unique HH ID in wave 2,Region Code,Zone Code,Woreda Code,Kebele/FA Code,Parcel ID,Field ID,Crop Name,How much [CROP] did you harvest from this [FIELD](QUANTITY),How much [CROP] did you harvest from this [FIELD](UNIT),household_id2,Area,Area (decimal),EA Longitude (WGS84) Modified,EA Latitude (WGS84) Modified
0,1.010109e+16,1.0,1.0,1.0,16.0,1,2,MAIZE,12.0,181.0,NaN,NaN,NaN,37.890876,14.353816
1,1.010109e+16,1.0,1.0,1.0,16.0,1,3,RED PEPPER,2.0,1.0,1.010109e+16,8.0,30.0,37.890876,14.353816
2,1.010109e+16,1.0,1.0,1.0,16.0,1,4,SORGHUM,350.0,1.0,NaN,NaN,NaN,37.890876,14.353816
3,1.010109e+16,1.0,1.0,1.0,16.0,1,2,MAIZE,30.0,181.0,NaN,NaN,NaN,NaN,NaN
4,1.010109e+16,1.0,1.0,1.0,16.0,1,3,RED PEPPER,24.0,181.0,NaN,NaN,NaN,NaN,NaN


In [33]:
c_conversions = c_conversions.drop_duplicates(subset=['Crop Name','Unit Code'])

Add conversion factor to yields dataframe:

In [34]:
if year in [2013, 2015]:
    c_yields = pd.merge(c_yields,
                    c_conversions,
                    how='left',
                    left_on=['Crop Name','How much [CROP] did you harvest from this [FIELD](UNIT)'],
                    right_on=['Crop Name','Unit Code'])

Perform conversion:

In [35]:
if year in [2013, 2015]:
    c_yields['Quantity_Kilograms'] = c_yields['How much [CROP] did you harvest from this [FIELD](QUANTITY)'] * c_yields['Conversion Factor (National)']
elif year == 2011:
    c_yields['Quantity_Kilograms'] = c_yields['How much crop harvested from this field during the last season KILOS']

In [36]:
c_yields

,Unique HH ID in wave 2,Region Code,Zone Code,Woreda Code,Kebele/FA Code,Parcel ID,Field ID,Crop Name,How much [CROP] did you harvest from this [FIELD](QUANTITY),How much [CROP] did you harvest from this [FIELD](UNIT),household_id2,Area,Area (decimal),EA Longitude (WGS84) Modified,EA Latitude (WGS84) Modified,Unit Code,Unit Name,Conversion Factor (National),Quantity_Kilograms
0,1.010109e+16,1.0,1.0,1.0,16.0,1,2,MAIZE,12.0,181.0,NaN,NaN,NaN,37.890876,14.353816,181.0,Tasa/Tanika/Shember/Selemon Small,0.438,5.256
1,1.010109e+16,1.0,1.0,1.0,16.0,1,3,RED PEPPER,2.0,1.0,1.010109e+16,8.0,30.0,37.890876,14.353816,1.0,Kilogram,1.000,2.000
2,1.010109e+16,1.0,1.0,1.0,16.0,1,4,SORGHUM,350.0,1.0,NaN,NaN,NaN,37.890876,14.353816,1.0,Kilogram,1.000,350.000
3,1.010109e+16,1.0,1.0,1.0,16.0,1,2,MAIZE,30.0,181.0,NaN,NaN,NaN,NaN,NaN,181.0,Tasa/Tanika/Shember/Selemon Small,0.438,13.140
4,1.010109e+16,1.0,1.0,1.0,16.0,1,3,RED PEPPER,24.0,181.0,NaN,NaN,NaN,NaN,NaN,181.0,Tasa/Tanika/Shember/Selemon Small,0.110,2.640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25919,1.501021e+17,15.0,1.0,2.0,28.0,3,1,COFFEE,25.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,Kilogram,1.000,25.000
25920,1.501021e+17,15.0,1.0,2.0,28.0,4,2,CHAT,3.0,192.0,NaN,NaN,NaN,NaN,NaN,192.0,Zorba/Akara Medium,0.270,0.810
25921,1.501021e+17,15.0,1.0,2.0,28.0,3,1,BANANAS,2.0,42.0,NaN,NaN,NaN,NaN,NaN,42.0,Bunch Medium,17.500,35.000
25922,1.501021e+17,15.0,1.0,2.0,28.0,3,1,COFFEE,10.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,Kilogram,1.000,10.000


In [37]:
c_yields.shape

(25924, 19)

### Land Area Lookups

In [38]:
if year == 2015:
    int_path = "Land Area Conversion Factor/"
    f = "ET_local_area_unit_conversion.csv"
elif year == 2013:
    int_path = ""
    f = "ET_local_area_unit_conversion.dta"
elif year == 2011:
    int_path = ""
    f = "ET_local_area_unit_conversion.csv"    

if year == 2013:
    land_conversions = pd.read_stata(f'{LSMS_PATH}/{int_path}{f}')
else:
    land_conversions = pd.read_csv(f'{LSMS_PATH}/{int_path}{f}')    

Remove duplicates in land conversions dataframe:

In [39]:
land_conversions = land_conversions.drop_duplicates(subset=['region','zonename','woredaname'])

In [40]:
def gen_reg_zone_woreda(row, f):
    """
    Generates a string combining region_zone_woreda for use during lookups.
    """
    if f == 'c_yields':
        region = row['Region Code']
        zone = int(row['Zone Code'])
        woreda = int(row['Woreda Code'])
        return f"{region}_{zone}_{woreda}".lower()
    elif f == 'land_conversions':
        region = row['region']
        zone = int(row['zone'])
        woreda = int(row['woreda'])
        return f"{region}_{zone}_{woreda}".lower()

In [41]:
c_yields['region_zone_woreda'] = c_yields.apply(lambda row: gen_reg_zone_woreda(row, 'c_yields'), axis=1)

In [42]:
land_conversions['region_zone_woreda'] = land_conversions.apply(lambda row: gen_reg_zone_woreda(row, 'land_conversions'), axis=1)

In [43]:
c_yields.shape

(25924, 20)

In [44]:
c_yields = c_yields.merge(land_conversions, on='region_zone_woreda', how='left')

In [45]:
c_yields.shape

(25924, 27)

In [46]:
c_yields['Area_Normalized'] = c_yields['Area'] * c_yields['conversion']

### Subset Crop Yields

In [47]:
c_yields_out = c_yields[['Unique HH ID in wave 2','region_zone_woreda','Crop Name','Quantity_Kilograms','EA Longitude (WGS84) Modified','EA Latitude (WGS84) Modified','Area_Normalized']]

In [48]:
out_vars = {'Unique HH ID in wave 2': 'Household_UUID',
            'zonename': 'Zone',
            'woredaname': 'Woreda',
            'Crop Name': 'Crop Name',
            'Quantity_Kilograms': 'Quantity (kg)',
            'EA Longitude (WGS84) Modified': 'Lon',
            'EA Latitude (WGS84) Modified': 'Lat',
            'Area_Normalized': 'Area (Normalized)'}

In [49]:
c_yields_out = c_yields_out.rename(columns=out_vars)
c_yields_out['Crop Name'] = c_yields_out['Crop Name'].apply(lambda x: str(x).title())

In [50]:
c_yields_out['Year'] = year

In [51]:
c_yields_out

,Household_UUID,region_zone_woreda,Crop Name,Quantity (kg),Lon,Lat,Area (Normalized),Year
0,1.010109e+16,1.0_1_1,Maize,5.256,37.890876,14.353816,NaN,2015
1,1.010109e+16,1.0_1_1,Red Pepper,2.000,37.890876,14.353816,NaN,2015
2,1.010109e+16,1.0_1_1,Sorghum,350.000,37.890876,14.353816,NaN,2015
3,1.010109e+16,1.0_1_1,Maize,13.140,NaN,NaN,NaN,2015
4,1.010109e+16,1.0_1_1,Red Pepper,2.640,NaN,NaN,NaN,2015
...,...,...,...,...,...,...,...,...
25919,1.501021e+17,15.0_1_2,Coffee,25.000,NaN,NaN,NaN,2015
25920,1.501021e+17,15.0_1_2,Chat,0.810,NaN,NaN,NaN,2015
25921,1.501021e+17,15.0_1_2,Bananas,35.000,NaN,NaN,NaN,2015
25922,1.501021e+17,15.0_1_2,Coffee,10.000,NaN,NaN,NaN,2015


## Geocode to GADM3

In [52]:
admin3 = gpd.read_file("GADM3/gadm36_ETH_3.shp")
admin3['country'] = admin3['NAME_0']
admin3['state'] = admin3['NAME_1']
admin3['admin1'] = admin3['NAME_1']
admin3['admin2'] = admin3['NAME_2']
admin3['admin3'] = admin3['NAME_3']
admin3 = admin3[['geometry','country','state','admin1','admin2','admin3']]

In [53]:
c_yields_out['geometry'] = c_yields_out.apply(lambda x: Point(x.Lon, x.Lat), axis=1)
c_yields_out = c_yields_out.dropna(subset=["Lon", "Lat"])
gdf = gpd.GeoDataFrame(c_yields_out, crs='epsg:4326')

In [54]:
# Spatial merge on GADM to obtain admin areas
gdf = gpd.sjoin(gdf, admin3, how="left", op='intersects')

In [55]:
gdf = gdf[['Year','country', 'state', 'admin1', 'admin2', 'admin3', 'Household_UUID', 'Crop Name', 'Quantity (kg)',
       'Lon', 'Lat', 'Area (Normalized)']]

In [56]:
gdf.head()

,Year,country,state,admin1,admin2,admin3,Household_UUID,Crop Name,Quantity (kg),Lon,Lat,Area (Normalized)
0,2015,Ethiopia,Tigray,Tigray,Semien Mi'irabaw,Tahtay Adiyabo,1.010109e+16,Maize,5.256,37.890876,14.353816,NaN
1,2015,Ethiopia,Tigray,Tigray,Semien Mi'irabaw,Tahtay Adiyabo,1.010109e+16,Red Pepper,2.000,37.890876,14.353816,NaN
2,2015,Ethiopia,Tigray,Tigray,Semien Mi'irabaw,Tahtay Adiyabo,1.010109e+16,Sorghum,350.000,37.890876,14.353816,NaN
10,2015,Ethiopia,Tigray,Tigray,Semien Mi'irabaw,Tahtay Adiyabo,1.010109e+16,Maize,8.760,37.890876,14.353816,NaN
11,2015,Ethiopia,Tigray,Tigray,Semien Mi'irabaw,Tahtay Adiyabo,1.010109e+16,Sesame,1.500,37.890876,14.353816,NaN


In [57]:
gdf.to_csv(f'Results/LSMS_{year}.csv', index=False)